<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/JARVIS_QuantumEspressoColab_Basic_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Run Quantum espresso calculations with JARVIS-Tools

In [1]:
!pip install jarvis-tools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.3/515.3 kB 29.9 MB/s eta 0:00:00


In [2]:
%%time
import os
# Clone the QE git
if not os.path.exists('q-e'):
   os.system('git clone https://github.com/QEF/q-e.git')
# Install the dependencies
!apt-get install -y libfftw3-3 libfftw3-dev libfftw3-doc
%cd q-e
# run the configure file
!DFLAGS='-D__OPENMP -D__FFTW3 -D__MPI -D__SCALAPACK' FFT_LIBS='-lfftw3'  ./configure --enable-openmp
! make pw

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libfftw3-bin libfftw3-long3 libfftw3-quad3 libfftw3-single3
The following NEW packages will be installed:
  libfftw3-3 libfftw3-bin libfftw3-dev libfftw3-doc libfftw3-long3
  libfftw3-quad3 libfftw3-single3
0 upgraded, 7 newly installed, 0 to remove and 38 not upgraded.
Need to get 4,251 kB of archives.
After this operation, 23.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 libfftw3-long3 amd64 3.3.8-2ubuntu1 [313 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/main amd64 libfftw3-single3 amd64 3.3.8-2ubuntu1 [756 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 libfftw3-3 amd64 3.3.8-2ubuntu1 [1,760 B]
Get:4 http://archive.ubuntu.com/ubuntu focal/main amd64 libfftw3-quad3 amd64 3.3.8-2ubuntu1 [673 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal/main amd64 libfftw3-bin

In [3]:
#!make ph

In [4]:
!lscpu 

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          2
On-line CPU(s) list:             0,1
Thread(s) per core:              2
Core(s) per socket:              1
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           79
Model name:                      Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:                        0
CPU MHz:                         2199.998
BogoMIPS:                        4399.99
Hypervisor vendor:               KVM
Virtualization type:             full
L1d cache:                       32 KiB
L1i cache:                       32 KiB
L2 cache:                        256 KiB
L3 cache:                        55 MiB
NUMA node0 CPU(s):               0,1
Vulnerability 

In [6]:
from jarvis.core.atoms import Atoms
from jarvis.db.figshare import data
import numpy as np
import pandas as pd

dft_3d = data('dft_3d')
df=pd.DataFrame(dft_3d)
#Filter based on elements
def has_elements(atoms_dict = {}, my_element = ['Al','O']):
    atoms = Atoms.from_dict(atoms_dict)
    elements = atoms.elements
    if set(my_element)==set(elements):
        return True
    else:
        return np.nan
    
df['el'] = df['atoms'].apply(lambda x: has_elements(atoms_dict = x, my_element = ['Si']))
df1 = df.dropna()
df_eform_filter = df1[df1['formation_energy_peratom']==0]
print(df_eform_filter)



Obtaining 3D dataset 76k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815699
Loading the zipfile...
Loading completed.
               jid spg_number spg_symbol formula  formation_energy_peratom  \
25912  JVASP-24792        227      Fd-3m      Si                       0.0   
42489   JVASP-1002        227      Fd-3m      Si                      -0.0   

            func  optb88vdw_bandgap  \
25912  OptB88vdW              0.731   
42489  OptB88vdW              0.731   

                                                   atoms   slme  \
25912  {'lattice_mat': [[3.3641546699579417, -2.54062...  33.23   
42489  {'lattice_mat': [[3.3641499856336465, -2.50271...  33.23   

      magmom_oszicar  ... poisson  \
25912            0.0  ...      na   
42489            0.0  ...    0.21   

                                               raw_files nat bulk_modulus_kv  \
25912  [DFPT,JVASP-24792.zip,https://ndownloader.figs...   2  

In [7]:
#Lets select JVASP-1002
from jarvis.db.figshare import get_jid_data
from jarvis.tasks.qe.super import SuperCond
from jarvis.core.utils import get_factors
from jarvis.core.atoms import Atoms
from jarvis.db.figshare import data, get_jid_data
from jarvis.core.kpoints import Kpoints3D
from jarvis.tasks.queue_jobs import Queue
from jarvis.db.jsonutils import dumpjson
import os
from jarvis.analysis.structure.spacegroup import Spacegroup3D
import glob
from jarvis.db.jsonutils import loadjson

jid = 'JVASP-1002' #Silicon
dat = get_jid_data(jid=jid, dataset="dft_3d")

Obtaining 3D dataset 76k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815699
Loading the zipfile...
Loading completed.


In [8]:
def non_prime_kpoints(kpts=[]):
    """Get non prime kpoints (used for supercon calculations)."""
    mem = []
    for i in kpts:
        facts = get_factors(i)
        if len(facts) == 1:
            val = i + 1
        else:
            val = i
        mem.append(val)
    return mem

qe_cmd = "/content/q-e/bin/pw.x"
a_atoms = Atoms.from_dict(dat["atoms"])
atoms = Spacegroup3D(a_atoms).refined_atoms.get_primitive_atoms
kp = Kpoints3D().automatic_length_mesh(lattice_mat=atoms.lattice_mat, length=dat["kpoint_length_unit"])
kpts = kp._kpoints[0]
kpts=non_prime_kpoints(kpts)
kp=Kpoints3D(kpoints=[kpts])
print ('kpts',kpts)


kpts [12, 12, 12]


In [9]:
from jarvis.tasks.qe.qe import QEjob
relax = {
            "control": {
                # "calculation": "'scf'",
                "calculation":  "'vc-relax'",
                "restart_mode": "'from_scratch'",
                "prefix": "'RELAX'",
                "outdir": "'./'",
                "tstress": ".true.",
                "tprnfor": ".true.",
                "disk_io": "'nowf'",
                "wf_collect": ".true.",
                "pseudo_dir": None,
                "verbosity": "'high'",
                "nstep": 100,
            },
            "system": {
                "ibrav": 0,
                "nat": None,
                "ntyp": None,
                "ecutwfc": 45,
                "ecutrho": 250,
                "q2sigma": 1,
                "ecfixed": 44.5,
                "qcutz": 800,
                "occupations": "'smearing'",
                "degauss": 0.01,
                "lda_plus_u": ".false.",
            },
            "electrons": {
                "diagonalization": "'david'",
                "mixing_mode": "'local-TF'",
                "mixing_beta": 0.3,
                "conv_thr": "1d-9",
            },
            "ions": {"ion_dynamics": "'bfgs'"},
            "cell": {"cell_dynamics": "'bfgs'", "cell_dofree": "'all'"},
        }
qejob_relax = QEjob(
    atoms=a_atoms,
    input_params=relax,
    output_file="relax.out",
    qe_cmd=qe_cmd,
    jobname="relax",
    kpoints=kp,
    input_file="arelax.in",
    url=None,
    psp_dir=None,
    psp_temp_name=None,
)

Please cite for PSPs:
https://doi.org/10.1016/j.commatsci.2013.08.053


In [10]:
%%time
info = qejob_relax.runjob()

Sanitizing Atoms.
cmd /content/q-e/bin/pw.x<arelax.in
HERE1
HERE2 /content/q-e/RELAX.save/data-file-schema.xml
CPU times: user 3.47 s, sys: 443 ms, total: 3.92 s
Wall time: 10min 54s


In [11]:
!ls

archive		 dft-d3			KCW	    PHonon	   std.err
arelax.in	 Doc			KS_Solvers  PP		   TDDFPT
atomic		 environment_variables	LAXlib	    pseudo	   test-suite
bin		 EPW			License     PW		   upflib
cmake		 external		logo.jpg    PWCOND	   UtilXlib
CMakeLists.txt	 FFTXlib		LR_Modules  QEHeat	   XClib
configure	 GUI			Makefile    README_GPU.md  XSpectra
CONTRIBUTING.md  GWW			make.inc    README.md
COUPLE		 HP			MBD	    relax.json
CPV		 include		Modules     relax.out
dev-tools	 install		NEB	    RELAX.save


In [12]:
info

{'out_path': '/content/q-e/relax.out',
 'xml_path_': 'na',
 'total_energy': -260.1408474992857,
 'job_done': True,
 'xml_path': '/content/q-e/RELAX.save/data-file-schema.xml'}

In [13]:
from jarvis.io.qe.outputs import DataFileSchema
ds = DataFileSchema('/content/q-e/RELAX.save/data-file-schema.xml')

In [14]:
print(ds.final_energy) #Hartree, QE usually reports data in Ha

-9.560000639789795


In [15]:
print(ds.final_energy_breakdown) #eV

{'etot': -260.14084749356573, 'eband': 8.436527609568326, 'ehart': 14.899778826580325, 'vtxc': -90.79505588787835, 'etxc': -116.00481040497618, 'ewald': -228.46781748029127, 'demet': -2.157125763338667e-05}


In [16]:
print(ds.final_structure)

Si2
1.0
3.3265423990964518 -3.2808606597162537e-09 1.920580147535111
1.10884746327226 3.136294422455583 1.920580147535111
-1.4132770633561725e-19 6.7874545599611735e-18 3.8411602950702224
Si
2
Cartesian
3.880966126640198 2.7442575742219284 6.7220305163728895
0.5544237357285162 0.39203684495279384 0.9602900737675555



In [17]:
print(ds.indir_gap)

0.46651330249232803


In [18]:
print(ds.nelec)

8


In [19]:
print(ds.efermi)

6.361279608518402


In [20]:
#Other data available in xml file
import pprint
pprint.pprint(ds.data)

Streaming output truncated to the last 5000 lines.
                                                                                          '3.555352731062077E-01   '
                                                                                          '4.173112583550452E-01   '
                                                                                          '4.354861317379202E-01',
                                                                                 '@size': '8'},
                                                                 'k_point': {'#text': '4.379080664274246E-01   '
                                                                                      '1.032159158340963E-01  '
                                                                                      '-2.528263402437891E-01',
                                                                             '@weight': '4.6296296296296545E-003'},
                                                   